<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Visualización de Datos</p>
<p style="margin: 0; text-align:right;">Máster Universitario de Ciencia de datos </p>
<p style="margin: 0; text-align:right;">Universitat Oberta de Catalunya</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Autor: Jorge Alonso Hernández</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Práctica 2: Creación de la visualización y entrega del proyecto

Carga inicial de librerías

In [1]:
#Carga de librerías
import pandas as pd
import numpy as np 
import itertools

from datetime import datetime

Lectura de los datos de ficheros iniciales

In [2]:
data_covid = pd.read_csv("datasets/covid_canarias.csv")
municipios = pd.read_csv("datasets/municipios.csv")
islas = pd.read_csv("datasets/islas.csv")

Visualizamos una muestra de los datos

In [3]:
data_covid.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso,identificacion_caso,fallecido,curado,continua_enfermo,provincia
0,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
1,14/03/2022,TENERIFE,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
2,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,06/01/2022,NaN,17/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
3,14/03/2022,TENERIFE,Adeje,Hombre,50 - 59,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
4,14/03/2022,TENERIFE,Adeje,Hombre,40 - 49,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife


Realizamos una transformación en los datos de forma que convertiremos los nombres de las islas en mayúsculas al formato real con cada inicial en mayúsculas sólo.

In [4]:
islas['isla'] = islas['isla'].str.title()
data_covid['isla'] = data_covid['isla'].str.title()

Realizaremos una transformación también en los datos de los municipios ya que podemos ver que existe una incoherencia entre los ficheros con los datos de covid y el de los municipios de forma que adaptaremos los nombres de los municipios a los que existen en el fichero de municipios.

In [5]:
#Obtenenmos los nombres de los municipios en el documento de casos de covid
municipios_ori = pd.DataFrame(pd.unique(data_covid['municipio']))
municipios_ori.columns = ['municipio']

#Comprobamos los nombres de los municipios
df_destino = municipios.loc[~municipios['municipio'].isin(pd.unique(data_covid['municipio']))]
df_destino = pd.DataFrame(df_destino['municipio'])
df_origen = municipios_ori.loc[~municipios_ori['municipio'].isin(municipios['municipio'])]

df_origen = df_origen.sort_values(by='municipio')
df_destino = df_destino.sort_values(by='municipio')

df_origen = df_origen.reset_index()
df_destino = df_destino.reset_index()

In [6]:
print(df_origen)

   index              municipio
0     45               El Pinar
1     81           Fuencaliente
2     70      Icod de Los Vinos
3      5              La Laguna
4     24      Puerto de La Cruz
5     87        SIN ESPECIFICAR
6     32  San juan de La Rambla
7     29    Santa María de Guía
8     67            Valsequillo
9     69               Vilaflor


In [7]:
print(df_destino)

   index                            municipio
0     87                El Pinar de El Hierro
1     47             Fuencaliente de La Palma
2     55                    Icod de los Vinos
3     61                    Puerto de la Cruz
4     56           San Cristóbal de La Laguna
5     67                San Juan de la Rambla
6     22  Santa María de Guía de Gran Canaria
7     30          Valsequillo de Gran Canaria
8     85                   Vilaflor de Chasna


In [8]:
#Eliminamos el valor SIN ESPECIFICAR pues es correcto que estos valores no se visualicen en el mapa 
df_origen = df_origen.drop(5).reset_index()

In [9]:
print(df_origen)

   level_0  index              municipio
0        0     45               El Pinar
1        1     81           Fuencaliente
2        2     70      Icod de Los Vinos
3        3      5              La Laguna
4        4     24      Puerto de La Cruz
5        6     32  San juan de La Rambla
6        7     29    Santa María de Guía
7        8     67            Valsequillo
8        9     69               Vilaflor


In [10]:
for i in range(0,len(df_origen)):
    data_covid = data_covid.replace(df_origen['municipio'][i], df_destino['municipio'][i])


Una vez realizadas las transformaciones de los datos inicializaremos la obtención de los conjuntos de datos que nos van a permitir visualizar los resultados. Inicialmente realizamos una separación entre los casos registrados en función del lugar en el que se ha realizado la asignación del caso y los casos en función del lugar de residencia del paciente.

In [11]:
asignacion = data_covid[data_covid['identificacion_caso'] == 'Asignación']
residencia = data_covid[data_covid['identificacion_caso'] == 'Residencia']

In [12]:
asignacion.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso,identificacion_caso,fallecido,curado,continua_enfermo,provincia
0,14/03/2022,Tenerife,Adeje,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
1,14/03/2022,Tenerife,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
2,14/03/2022,Tenerife,Adeje,Hombre,40 - 49,06/01/2022,NaN,17/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
3,14/03/2022,Tenerife,Adeje,Hombre,50 - 59,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife
4,14/03/2022,Tenerife,Adeje,Hombre,40 - 49,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Asignación,No,Sí,No,Santa Cruz de Tenerife


In [13]:
residencia.head()

,fecha_datos,isla,municipio,sexo,grupo_edad,fecha_caso,fecha_fallecido,fecha_curado,estado_caso,identificacion_caso,fallecido,curado,continua_enfermo,provincia
282989,14/03/2022,Tenerife,Adeje,Mujer,20 - 29,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Residencia,No,Sí,No,Santa Cruz de Tenerife
282990,14/03/2022,Tenerife,SIN ESPECIFICAR,Mujer,20 - 29,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Residencia,No,Sí,No,Santa Cruz de Tenerife
282991,14/03/2022,Tenerife,Adeje,Mujer,10 - 19,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Residencia,No,Sí,No,Santa Cruz de Tenerife
282992,14/03/2022,Tenerife,SIN ESPECIFICAR,Hombre,30 - 39,02/01/2022,NaN,13/01/2022,Cerrado por alta médica,Residencia,No,Sí,No,Santa Cruz de Tenerife
282993,14/03/2022,Tenerife,Adeje,Hombre,30 - 39,08/01/2022,NaN,19/01/2022,Cerrado por alta médica,Residencia,No,Sí,No,Santa Cruz de Tenerife


Obtenemos los datos de los municipios, islas y provincias por separado de forma que nos ayuden a obtener los datos en función de cada una de estas categorías.

In [14]:
municipios = municipios.merge(islas[['geocode', 'isla', 'provincia', 'poblacion']],  left_on='geocode_isla', right_on='geocode')

In [15]:
df_municipios = pd.DataFrame(municipios['municipio'])
df_islas = pd.DataFrame(islas['isla'])
df_provincias = pd.DataFrame(pd.unique(islas['provincia']))
df_provincias.columns = ['provincia']

In [16]:
df_municipios

,municipio
0,Agaete
1,Agüimes
2,Artenara
3,Arucas
4,Firgas
...,...
83,Tijarafe
84,Villa de Mazo
85,Frontera
86,Valverde


In [17]:
df_islas

,isla
0,El Hierro
1,Fuerteventura
2,Gran Canaria
3,La Gomera
4,La Palma
5,Lanzarote
6,Tenerife


In [18]:
df_provincias

,provincia
0,Santa Cruz de Tenerife
1,Las Palmas


También obtenemos los datos referentes a las capitales de las islas y de las provincias para la visualización de estas en el mapa.

In [19]:
capitales = ['Santa Cruz de Tenerife', 'Santa Cruz de La Palma', 'San Sebastián de La Gomera', 'Valverde', 'Las Palmas de Gran Canaria', 'Puerto del Rosario', 'Arrecife']
capitales_provincia = ['Santa Cruz de Tenerife', 'Las Palmas de Gran Canaria']

df_capitales = municipios.loc[municipios['municipio'].isin(capitales)]
df_capitales = df_capitales.reset_index()

df_capitales_provincia = municipios.loc[municipios['municipio'].isin(capitales_provincia)]
df_capitales_provincia = df_capitales_provincia.reset_index()

In [20]:
df_capitales = df_capitales[['municipio','isla', 'provincia','longitud_capital','latitud_capital', 'poblacion']]
df_capitales_provincia = df_capitales_provincia[['municipio','isla', 'provincia','longitud_capital','latitud_capital']]

In [21]:
df_capitales

,municipio,isla,provincia,longitud_capital,latitud_capital,poblacion
0,Las Palmas de Gran Canaria,Gran Canaria,Las Palmas,-15.428533,28.125058,855521
1,Puerto del Rosario,Fuerteventura,Las Palmas,-13.859694,28.498774,119732
2,Arrecife,Lanzarote,Las Palmas,-13.546719,28.959235,155812
3,Santa Cruz de Tenerife,Tenerife,Santa Cruz de Tenerife,-16.254634,28.469805,928604
4,San Sebastián de La Gomera,La Gomera,Santa Cruz de Tenerife,-17.109706,28.090378,21678
5,Santa Cruz de La Palma,La Palma,Santa Cruz de Tenerife,-17.764744,28.683243,83458
6,Valverde,El Hierro,Santa Cruz de Tenerife,-17.914964,27.806884,11147


In [22]:
df_capitales_provincia

,municipio,isla,provincia,longitud_capital,latitud_capital
0,Las Palmas de Gran Canaria,Gran Canaria,Las Palmas,-15.428533,28.125058
1,Santa Cruz de Tenerife,Tenerife,Santa Cruz de Tenerife,-16.254634,28.469805


Realizaremos ahora la obtención de los datos finales, se obtendrán por un lado los datos de los nuevos casos de cada uno de los días y a la vez se realizará un conteo de los datos de casos que se van acumulando día a día de forma que se tenga un registro total. Para ello realizamos una función que se va a encargar de realizar el conteo de los casos en función de la fecha y del tipo de agrupamiento que puede ser por municipios, islas o provincias. Una vez realizado el conteo por fecha y tipo de agrupación se irá recorriendo el dataframe resultante y se irán trasponiendo los datos de forma que se obtenga una columna de dataframe por fecha con sus resultados asociados.

In [23]:
def conversion(df, tipo_agrupamiento):
    # Dataframes resultantes
    df_result = pd.DataFrame()
    df_result_accumul = pd.DataFrame()
        
    if tipo_agrupamiento == 'municipios':
        df_agrupamientos = df_municipios
        label = 'municipio'
    elif tipo_agrupamiento == 'islas':
        df_agrupamientos = df_islas
        label = 'isla'
    elif tipo_agrupamiento == 'provincias':
        df_agrupamientos = df_provincias
        label = 'provincia'
    else:
        return None
    
    # El método irá recorriendo cada uno de los municipios/islas/provincias y trasponiendo las filas por columnas para obtener el formato deseado
    for index, row in df_agrupamientos.iterrows():
        aux = df[df[label] == row[label]]
        aux = aux.groupby(['fecha_caso',label]).size().reset_index()
        aux.columns=['fecha', label, row[label]]
        aux = aux[['fecha', row[label]]]
        aux['fecha'] = pd.to_datetime(aux['fecha'], dayfirst=True,format='%d/%m/%Y', exact=True)
        aux['fecha'] = aux['fecha'].dt.strftime('%d/%m/%Y')
        
        #Ordenamos los resultados
        aux = aux.sort_values(by='fecha')
        aux = aux.set_index('fecha')
        aux_acc = aux.cumsum()
        aux = aux.transpose()
        aux_acc = aux_acc.transpose()
        
        df_result = pd.concat([df_result, aux], axis = 0)
        df_result_accumul = pd.concat([df_result_accumul, aux_acc], axis = 0)
    
    #Se realiza una segunda ordenación ya que el método concat de pandas añade al finalRealizamos la obtención de los datos empleando el método definido, obtenemos en primer lugar los datos en función de la asignación del caso. las columnas que no están en un dataframe con valor Nan
    
    # Ordenamos por fecha y cumplimentamos los NaN con el valor anterior
    df_result_accumul = df_result_accumul.transpose()
    df_result_accumul = df_result_accumul.sort_values(by=['fecha']) # Se ordena de nuevo para que las fechas que se han añadido con valor NaN
    df_result_accumul[0:1] = df_result_accumul[0:1].fillna(0) # Si los valores NaN están al incio se cumplimentan con 0
    df_result_accumul = df_result_accumul.fillna(method="pad")# Asignamos el valor inmediatamente anterior a los NaN
    df_result_accumul = df_result_accumul.transpose()
    
    #Ordenamos por fecha y cumplimentamos los NaN por 0
    df_result = df_result.replace(np.nan, 0)
    df_result = df_result.transpose()
    df_result = df_result.sort_values(by=['fecha'])
    df_result = df_result.transpose()
    
    df_result = df_result.reset_index()
    df_result_accumul = df_result_accumul.reset_index()
    
    return df_result, df_result_accumul

Realizamos la obtención de los datos empleando el método definido, obtenemos en primer lugar los datos en función de la asignación del caso.

In [24]:
data_municipios_asignacion, data_municipios_asignacion_accumul =  conversion(asignacion, 'municipios')
data_islas_asignacion, data_islas_asignacion_accumul = conversion(asignacion, 'islas')
data_provincias_asignacion, data_provincias_asignacion_accumul = conversion(asignacion, 'provincias')

Obtenemos los datos en función de la residencia del paciente.

In [25]:
data_municipios_residencia, data_municipios_residencia_accumul =  conversion(residencia, 'municipios')
data_islas_residencia, data_islas_residencia_accumul = conversion(residencia, 'islas')
data_provincias_residencia, data_provincias_residencia_accumul = conversion(residencia, 'provincias')

Añadimos los datos de isla y provincia a los datos de los municipios para la visualización.

In [26]:
data_municipios_asignacion = data_municipios_asignacion.merge(municipios[['municipio', 'isla', 'provincia']],  left_on='index', right_on='municipio')
data_municipios_asignacion_accumul = data_municipios_asignacion_accumul.merge(municipios[['municipio', 'isla', 'provincia']],  left_on='index', right_on='municipio')

data_municipios_residencia = data_municipios_residencia.merge(municipios[['municipio', 'isla', 'provincia']],  left_on='index', right_on='municipio')
data_municipios_residencia_accumul = data_municipios_residencia_accumul.merge(municipios[['municipio', 'isla', 'provincia']],  left_on='index', right_on='municipio')

Realizamos la visualización de los datos obtenidos y que serán visualizados. en primer lugar, los datos en función de la asignación del caso.

Datos por municipios.

In [27]:
#Datos por día
data_municipios_asignacion

,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021,municipio,isla,provincia
0,Agaete,0.0,0.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,1.0,Agaete,Gran Canaria,Las Palmas
1,Agüimes,0.0,3.0,0.0,4.0,0.0,6.0,0.0,0.0,1.0,...,9.0,1.0,0.0,0.0,0.0,0.0,5.0,Agüimes,Gran Canaria,Las Palmas
2,Artenara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Artenara,Gran Canaria,Las Palmas
3,Arucas,0.0,4.0,1.0,11.0,2.0,2.0,0.0,0.0,0.0,...,7.0,1.0,0.0,0.0,0.0,1.0,17.0,Arucas,Gran Canaria,Las Palmas
4,Firgas,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,3.0,Firgas,Gran Canaria,Las Palmas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Tijarafe,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,Tijarafe,La Palma,Santa Cruz de Tenerife
84,Villa de Mazo,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,Villa de Mazo,La Palma,Santa Cruz de Tenerife
85,Frontera,0.0,1.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,1.0,0.0,0.0,4.0,Frontera,El Hierro,Santa Cruz de Tenerife
86,Valverde,0.0,0.0,6.0,3.0,0.0,5.0,0.0,6.0,0.0,...,17.0,0.0,0.0,0.0,0.0,0.0,19.0,Valverde,El Hierro,Santa Cruz de Tenerife


In [28]:
#Datos acumulados
data_municipios_asignacion_accumul

,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021,municipio,isla,provincia
0,Agaete,0.0,0.0,0.0,6.0,6.0,7.0,7.0,7.0,7.0,...,388.0,388.0,388.0,388.0,388.0,388.0,389.0,Agaete,Gran Canaria,Las Palmas
1,Agüimes,0.0,3.0,3.0,7.0,7.0,13.0,13.0,13.0,14.0,...,1035.0,1036.0,1036.0,1036.0,1036.0,1036.0,1041.0,Agüimes,Gran Canaria,Las Palmas
2,Artenara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,53.0,53.0,53.0,53.0,53.0,53.0,53.0,Artenara,Gran Canaria,Las Palmas
3,Arucas,0.0,4.0,5.0,16.0,18.0,20.0,20.0,20.0,20.0,...,801.0,802.0,802.0,802.0,802.0,803.0,820.0,Arucas,Gran Canaria,Las Palmas
4,Firgas,0.0,1.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,...,269.0,269.0,269.0,270.0,270.0,270.0,273.0,Firgas,Gran Canaria,Las Palmas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Tijarafe,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,74.0,74.0,74.0,74.0,74.0,74.0,79.0,Tijarafe,La Palma,Santa Cruz de Tenerife
84,Villa de Mazo,0.0,0.0,1.0,2.0,3.0,4.0,4.0,4.0,4.0,...,164.0,164.0,164.0,164.0,164.0,164.0,167.0,Villa de Mazo,La Palma,Santa Cruz de Tenerife
85,Frontera,0.0,1.0,1.0,8.0,8.0,8.0,8.0,8.0,8.0,...,367.0,367.0,367.0,368.0,368.0,368.0,372.0,Frontera,El Hierro,Santa Cruz de Tenerife
86,Valverde,0.0,0.0,6.0,9.0,9.0,14.0,14.0,20.0,20.0,...,1344.0,1344.0,1344.0,1344.0,1344.0,1344.0,1363.0,Valverde,El Hierro,Santa Cruz de Tenerife


Datos por islas.

In [29]:
#Datos por día
data_islas_asignacion

fecha,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,30/11/2021,30/12/2021,31/01/2021,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021
0,El Hierro,0.0,1.0,6.0,16.0,0.0,8.0,0.0,6.0,0.0,...,0.0,17.0,0.0,21.0,0.0,0.0,1.0,0.0,0.0,25.0
1,Fuerteventura,1.0,82.0,6.0,87.0,9.0,16.0,16.0,2.0,6.0,...,31.0,126.0,3.0,123.0,20.0,3.0,24.0,9.0,3.0,133.0
2,Gran Canaria,62.0,784.0,148.0,604.0,71.0,461.0,118.0,42.0,30.0,...,178.0,1540.0,99.0,545.0,109.0,19.0,294.0,69.0,23.0,1206.0
3,La Gomera,0.0,12.0,0.0,5.0,0.0,9.0,0.0,0.0,0.0,...,0.0,24.0,0.0,9.0,2.0,0.0,0.0,3.0,1.0,22.0
4,La Palma,0.0,18.0,2.0,71.0,5.0,55.0,3.0,0.0,1.0,...,19.0,216.0,0.0,189.0,11.0,0.0,5.0,6.0,5.0,223.0
5,Lanzarote,14.0,235.0,35.0,70.0,12.0,40.0,13.0,4.0,7.0,...,25.0,336.0,34.0,66.0,6.0,7.0,6.0,14.0,5.0,254.0
6,Tenerife,67.0,729.0,47.0,417.0,70.0,255.0,87.0,74.0,83.0,...,125.0,2471.0,20.0,412.0,97.0,48.0,333.0,56.0,19.0,1570.0


In [30]:
#Datos acumulados
data_islas_asignacion_accumul

fecha,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,30/11/2021,30/12/2021,31/01/2021,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021
0,El Hierro,0.0,1.0,7.0,23.0,23.0,31.0,31.0,37.0,37.0,...,1803.0,1820.0,1820.0,1841.0,1841.0,1841.0,1842.0,1842.0,1842.0,1867.0
1,Fuerteventura,1.0,83.0,89.0,176.0,185.0,201.0,217.0,219.0,225.0,...,13641.0,13767.0,13770.0,13893.0,13913.0,13916.0,13940.0,13949.0,13952.0,14085.0
2,Gran Canaria,62.0,846.0,994.0,1598.0,1669.0,2130.0,2248.0,2290.0,2320.0,...,102744.0,104284.0,104383.0,104928.0,105037.0,105056.0,105350.0,105419.0,105442.0,106648.0
3,La Gomera,0.0,12.0,12.0,17.0,17.0,26.0,26.0,26.0,26.0,...,1819.0,1843.0,1843.0,1852.0,1854.0,1854.0,1854.0,1857.0,1858.0,1880.0
4,La Palma,0.0,18.0,20.0,91.0,96.0,151.0,154.0,154.0,155.0,...,9693.0,9909.0,9909.0,10098.0,10109.0,10109.0,10114.0,10120.0,10125.0,10348.0
5,Lanzarote,14.0,249.0,284.0,354.0,366.0,406.0,419.0,423.0,430.0,...,18579.0,18915.0,18949.0,19015.0,19021.0,19028.0,19034.0,19048.0,19053.0,19307.0
6,Tenerife,67.0,796.0,843.0,1260.0,1330.0,1585.0,1672.0,1746.0,1829.0,...,123828.0,126299.0,126319.0,126731.0,126828.0,126876.0,127209.0,127265.0,127284.0,128854.0


Datos por provincias.

In [31]:
#Datos por día
data_provincias_asignacion

fecha,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,30/11/2021,30/12/2021,31/01/2021,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021
0,Santa Cruz de Tenerife,67,760,55,509,75,327,90,80,84,...,144,2728,20,631,110,48,339,65,25,1840
1,Las Palmas,77,1101,189,761,92,517,147,48,43,...,234,2002,136,734,135,29,324,92,31,1593


In [32]:
#Datos p
data_provincias_asignacion_accumul

fecha,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,30/11/2021,30/12/2021,31/01/2021,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021
0,Santa Cruz de Tenerife,67,827,882,1391,1466,1793,1883,1963,2047,...,137143,139871,139891,140522,140632,140680,141019,141084,141109,142949
1,Las Palmas,77,1178,1367,2128,2220,2737,2884,2932,2975,...,134964,136966,137102,137836,137971,138000,138324,138416,138447,140040


Visualizamos los datos obtenidos en función de la residencia del paciente

Datos por municipios.

In [33]:
#Datos por día
data_municipios_residencia

,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021,municipio,isla,provincia
0,Agaete,0.0,2.0,0.0,6.0,0.0,2.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,6.0,Agaete,Gran Canaria,Las Palmas
1,Agüimes,0.0,22.0,2.0,15.0,1.0,15.0,1.0,5.0,2.0,...,23.0,3.0,1.0,11.0,2.0,1.0,39.0,Agüimes,Gran Canaria,Las Palmas
2,Artenara,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Artenara,Gran Canaria,Las Palmas
3,Arucas,2.0,46.0,13.0,36.0,3.0,16.0,5.0,0.0,0.0,...,18.0,3.0,0.0,6.0,1.0,1.0,57.0,Arucas,Gran Canaria,Las Palmas
4,Firgas,2.0,17.0,1.0,2.0,0.0,4.0,0.0,0.0,2.0,...,2.0,1.0,0.0,1.0,0.0,1.0,12.0,Firgas,Gran Canaria,Las Palmas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Tijarafe,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,Tijarafe,La Palma,Santa Cruz de Tenerife
84,Villa de Mazo,0.0,3.0,1.0,18.0,1.0,2.0,0.0,0.0,0.0,...,31.0,0.0,0.0,0.0,0.0,0.0,10.0,Villa de Mazo,La Palma,Santa Cruz de Tenerife
85,Frontera,0.0,0.0,0.0,8.0,0.0,0.0,0.0,4.0,0.0,...,16.0,0.0,0.0,1.0,0.0,0.0,5.0,Frontera,El Hierro,Santa Cruz de Tenerife
86,Valverde,0.0,1.0,5.0,3.0,0.0,3.0,0.0,2.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,10.0,Valverde,El Hierro,Santa Cruz de Tenerife


In [34]:
#Datos acumulados
data_municipios_residencia_accumul

,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021,municipio,isla,provincia
0,Agaete,0.0,2.0,2.0,8.0,8.0,10.0,10.0,10.0,10.0,...,585.0,585.0,585.0,585.0,585.0,585.0,591.0,Agaete,Gran Canaria,Las Palmas
1,Agüimes,0.0,22.0,24.0,39.0,40.0,55.0,56.0,61.0,63.0,...,3784.0,3787.0,3788.0,3799.0,3801.0,3802.0,3841.0,Agüimes,Gran Canaria,Las Palmas
2,Artenara,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,73.0,73.0,73.0,73.0,73.0,73.0,73.0,Artenara,Gran Canaria,Las Palmas
3,Arucas,2.0,48.0,61.0,97.0,100.0,116.0,121.0,121.0,121.0,...,3641.0,3644.0,3644.0,3650.0,3651.0,3652.0,3709.0,Arucas,Gran Canaria,Las Palmas
4,Firgas,2.0,19.0,20.0,22.0,22.0,26.0,26.0,26.0,28.0,...,787.0,788.0,788.0,789.0,789.0,790.0,802.0,Firgas,Gran Canaria,Las Palmas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Tijarafe,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,280.0,280.0,280.0,280.0,280.0,280.0,289.0,Tijarafe,La Palma,Santa Cruz de Tenerife
84,Villa de Mazo,0.0,3.0,4.0,22.0,23.0,25.0,25.0,25.0,25.0,...,603.0,603.0,603.0,603.0,603.0,603.0,613.0,Villa de Mazo,La Palma,Santa Cruz de Tenerife
85,Frontera,0.0,0.0,0.0,8.0,8.0,8.0,8.0,12.0,12.0,...,667.0,667.0,667.0,668.0,668.0,668.0,673.0,Frontera,El Hierro,Santa Cruz de Tenerife
86,Valverde,0.0,1.0,6.0,9.0,9.0,12.0,12.0,14.0,14.0,...,862.0,862.0,862.0,862.0,862.0,862.0,872.0,Valverde,El Hierro,Santa Cruz de Tenerife


Datos por islas.

In [35]:
#Datos por día
data_islas_residencia

fecha,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,30/11/2021,30/12/2021,31/01/2021,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021
0,El Hierro,0.0,1.0,6.0,16.0,0.0,8.0,0.0,6.0,0.0,...,0.0,17.0,0.0,21.0,0.0,0.0,1.0,0.0,0.0,25.0
1,Fuerteventura,1.0,82.0,6.0,87.0,9.0,16.0,16.0,2.0,6.0,...,31.0,126.0,3.0,123.0,20.0,3.0,24.0,9.0,3.0,133.0
2,Gran Canaria,62.0,784.0,148.0,604.0,71.0,461.0,118.0,42.0,30.0,...,178.0,1540.0,99.0,545.0,109.0,19.0,294.0,69.0,23.0,1206.0
3,La Gomera,0.0,12.0,0.0,5.0,0.0,9.0,0.0,0.0,0.0,...,0.0,24.0,0.0,9.0,2.0,0.0,0.0,3.0,1.0,22.0
4,La Palma,0.0,18.0,2.0,71.0,5.0,55.0,3.0,0.0,1.0,...,19.0,216.0,0.0,189.0,11.0,0.0,5.0,6.0,5.0,223.0
5,Lanzarote,14.0,235.0,35.0,70.0,12.0,40.0,13.0,4.0,7.0,...,25.0,336.0,34.0,66.0,6.0,7.0,6.0,14.0,5.0,254.0
6,Tenerife,67.0,729.0,47.0,417.0,70.0,255.0,87.0,74.0,83.0,...,125.0,2471.0,20.0,412.0,97.0,48.0,333.0,56.0,19.0,1570.0


In [36]:
#Datos acumulados
data_islas_residencia_accumul

fecha,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,30/11/2021,30/12/2021,31/01/2021,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021
0,El Hierro,0.0,1.0,7.0,23.0,23.0,31.0,31.0,37.0,37.0,...,1803.0,1820.0,1820.0,1841.0,1841.0,1841.0,1842.0,1842.0,1842.0,1867.0
1,Fuerteventura,1.0,83.0,89.0,176.0,185.0,201.0,217.0,219.0,225.0,...,13641.0,13767.0,13770.0,13893.0,13913.0,13916.0,13940.0,13949.0,13952.0,14085.0
2,Gran Canaria,62.0,846.0,994.0,1598.0,1669.0,2130.0,2248.0,2290.0,2320.0,...,102744.0,104284.0,104383.0,104928.0,105037.0,105056.0,105350.0,105419.0,105442.0,106648.0
3,La Gomera,0.0,12.0,12.0,17.0,17.0,26.0,26.0,26.0,26.0,...,1819.0,1843.0,1843.0,1852.0,1854.0,1854.0,1854.0,1857.0,1858.0,1880.0
4,La Palma,0.0,18.0,20.0,91.0,96.0,151.0,154.0,154.0,155.0,...,9693.0,9909.0,9909.0,10098.0,10109.0,10109.0,10114.0,10120.0,10125.0,10348.0
5,Lanzarote,14.0,249.0,284.0,354.0,366.0,406.0,419.0,423.0,430.0,...,18579.0,18915.0,18949.0,19015.0,19021.0,19028.0,19034.0,19048.0,19053.0,19307.0
6,Tenerife,67.0,796.0,843.0,1260.0,1330.0,1585.0,1672.0,1746.0,1829.0,...,123828.0,126299.0,126319.0,126731.0,126828.0,126876.0,127209.0,127265.0,127284.0,128854.0


Datos por provincias.

In [37]:
#Datos por día
data_provincias_residencia

fecha,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,30/11/2021,30/12/2021,31/01/2021,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021
0,Santa Cruz de Tenerife,67,760,55,509,75,327,90,80,84,...,144,2728,20,631,110,48,339,65,25,1840
1,Las Palmas,77,1101,189,761,92,517,147,48,43,...,234,2002,136,734,135,29,324,92,31,1593


In [38]:
#Datos acumulados
data_provincias_residencia_accumul

fecha,index,01/01/2021,01/01/2022,01/02/2021,01/02/2022,01/03/2021,01/03/2022,01/04/2021,01/05/2021,01/06/2021,...,30/11/2021,30/12/2021,31/01/2021,31/01/2022,31/03/2021,31/05/2021,31/07/2021,31/08/2021,31/10/2021,31/12/2021
0,Santa Cruz de Tenerife,67,827,882,1391,1466,1793,1883,1963,2047,...,137143,139871,139891,140522,140632,140680,141019,141084,141109,142949
1,Las Palmas,77,1178,1367,2128,2220,2737,2884,2932,2975,...,134964,136966,137102,137836,137971,138000,138324,138416,138447,140040


Una vez obtenidos los datos realizamos la escritura en formato Excel para realizar la subida a la plataforma de Flourish.

Escritura de los datos en función de la asignación del caso.

In [39]:
data_provincias_asignacion.to_excel('data/asignacion/provincias_asignacion.xlsx', index=False, encoding="utf8")
data_provincias_asignacion_accumul.to_excel('data/asignacion/provincias_asignacion_acc.xlsx', index=False, encoding="utf8")
data_islas_asignacion.to_excel('data/asignacion/islas_asignacion.xlsx', index=False, encoding="utf8")
data_islas_asignacion_accumul.to_excel('data/asignacion/islas_asignacion_acc.xlsx', index=False, encoding="utf8")
data_municipios_asignacion.to_excel('data/asignacion/municipios_asignacion.xlsx', index=False, encoding="utf8")
data_municipios_asignacion_accumul.to_excel('data/asignacion/municipios_asignacion_acc.xlsx', index=False, encoding="utf8")

Escritura de los datos en función de la residencia del paciente.

In [40]:
data_provincias_residencia.to_excel('data/residencia/provincias_residencia.xlsx', index=False, encoding="utf8")
data_provincias_residencia_accumul.to_excel('data/residencia/provincias_residencia_acc.xlsx', index=False, encoding="utf8")
data_islas_residencia.to_excel('data/residencia/islas_residencia.xlsx', index=False, encoding="utf8")
data_islas_residencia_accumul.to_excel('data/residencia/islas_residencia_acc.xlsx', index=False, encoding="utf8")
data_municipios_residencia.to_excel('data/residencia/municipios_residencia.xlsx', index=False, encoding="utf8")
data_municipios_residencia_accumul.to_excel('data/residencia/municipios_residencia_acc.xlsx', index=False, encoding="utf8")

Por último realizamos la escritura de los datos de las capitales.

In [41]:
df_capitales.to_csv('mapas/capitales.csv', index=False, sep=",")
df_capitales_provincia.to_csv('mapas/capitales_provincia.csv', index=False, sep=",")